Below is an example of the sift algorithm, with the trilinear interpolation smoothing of a histogram tensor of the generate descriptors being implemented in hardware using our overlay, histogram_tensor

This overlay stream in inputs for the histogram tensor calculations, which are done on the board, and the resulting structure is stream back out and placed back into the software structure for further calculations

In [1]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/descriptor/design_1.bit')
dma = overlay.axi_dma_0
# dma?

In [2]:
# Converts a KeyPoint into a Dictionary
def convertKPToDict(KP):
    return {
        'angle': KP.angle,
        'class_id': KP.class_id,
        'octave': KP.octave,
        'point': KP.pt,
        'response': KP.response,
        'size': KP.size
    }

def convertDictToKP(D):
    return cv2.KeyPoint(*D['point'],
                   D['size'],
                   D['angle'],
                   D['response'],
                   D['octave'],
                   D['class_id'])

def KeyPointsToJSON(KPs):
    output_list = []
    for KP in KPs:
        output_list.append(convertKPToDict(KP))
    return output_list

def unpackOctave(keypoint):
    """Compute octave, layer, and scale from a keypoint
    """
    octave = keypoint.octave & 255
    layer = (keypoint.octave >> 8) & 255
    if octave >= 128:
        octave = octave | -128
    scale = 1 / np.float32(1 << octave) if octave >= 0 else np.float32(1 << -octave)
    return octave, layer, scale


In [3]:
from sfm_algo_unpacked import unpackOctave
from numpy import all, any, array, arctan2, cos, sin, exp, dot, log, logical_and, roll, sqrt, stack, trace, unravel_index, pi, deg2rad, rad2deg, where, zeros, floor, full, nan, isnan, round, float32
from pynq import allocate

def fpga_zip(row_array, col_array, mag_array, ori_array):
    minimum = min([len(array) for array in [row_array, col_array, mag_array, ori_array]])
    input_buf = []
    for i in range(minimum):
        #convert values to fixed point
        row_bin_fixed = np.float16(row_array[i])
        col_bin_fixed = np.float16(col_array[i])
        magnitude_fixed = np.float16(mag_array[i])
        orientation_bin_fixed = np.float16(ori_array[i])
        in_val = [row_bin_fixed, col_bin_fixed, magnitude_fixed, orientation_bin_fixed]
        input_buf.append(in_val)
    return input_buf

def generateDescriptors_hardware(keypoints, gaussian_images, dma, window_width=4, num_bins=8, scale_multiplier=3, descriptor_max_value=0.2):
    """Generate descriptors for each keypoint
    """
    print("Generating descriptors")
    descriptors = []

    for keypoint in keypoints:
        octave, layer, scale = unpackOctave(keypoint)
        gaussian_image = gaussian_images[octave + 1][layer]
        num_rows, num_cols = gaussian_image.shape
        point = round(scale * array(keypoint.pt)).astype('int')
        bins_per_degree = num_bins / 360.
        angle = 360. - keypoint.angle
        cos_angle = cos(deg2rad(angle))
        sin_angle = sin(deg2rad(angle))
        weight_multiplier = -0.5 / ((0.5 * window_width) ** 2)
        row_bin_list = []
        col_bin_list = []
        magnitude_list = []
        orientation_bin_list = []
        histogram_tensor = zeros((window_width + 2, window_width + 2, num_bins)).tolist()   # first two dimensions are increased by 2 to account for border effects

        # Descriptor window size (described by half_width)
        hist_width = scale_multiplier * 0.5 * scale * keypoint.size
        half_width = int(round(hist_width * sqrt(2) * (window_width + 1) * 0.5))   # sqrt(2) corresponds to diagonal length of a pixel
        half_width = int(min(half_width, sqrt(num_rows ** 2 + num_cols ** 2)))     # ensure half_width lies within image

        for row in range(-half_width, half_width + 1):
            for col in range(-half_width, half_width + 1):
                row_rot = col * sin_angle + row * cos_angle
                col_rot = col * cos_angle - row * sin_angle
                row_bin = (row_rot / hist_width) + 0.5 * window_width - 0.5
                col_bin = (col_rot / hist_width) + 0.5 * window_width - 0.5
                if row_bin > -1 and row_bin < window_width and col_bin > -1 and col_bin < window_width:
                    window_row = int(round(point[1] + row))
                    window_col = int(round(point[0] + col))
                    if window_row > 0 and window_row < num_rows - 1 and window_col > 0 and window_col < num_cols - 1:
                        dx = gaussian_image[window_row, window_col + 1] - gaussian_image[window_row, window_col - 1]
                        dy = gaussian_image[window_row - 1, window_col] - gaussian_image[window_row + 1, window_col]
                        gradient_magnitude = sqrt(dx * dx + dy * dy)
                        gradient_orientation = rad2deg(arctan2(dy, dx)) % 360
                        weight = exp(weight_multiplier * ((row_rot / hist_width) ** 2 + (col_rot / hist_width) ** 2))
                        row_bin_list.append(row_bin)
                        col_bin_list.append(col_bin)
                        magnitude_list.append(weight * gradient_magnitude)
                        orientation_bin_list.append((gradient_orientation - angle) * bins_per_degree)
        
        # do trilenear interpolation using the fpga overlay
        input_buf = fpga_zip(row_bin_list, col_bin_list, magnitude_list, orientation_bin_list)
        if (len(input_buf) == 0):
            print("Continuing")
            continue;
        input_buffer = allocate(shape=(len(input_buf), 4), dtype=np.float16)
        array_area = len(histogram_tensor) * len(histogram_tensor[0]) * len(histogram_tensor[0][0])
        output_buffer = allocate(shape=(array_area,), dtype=float32)
        print(input_buffer.physical_address)
        print(output_buffer.physical_address)
        input_buffer[:] = input_buf
        dma.sendchannel.transfer(input_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.transfer(output_buffer)
        dma.recvchannel.wait()
        descriptor_vector = fpga_unzip(output_buffer)
        del output_buffer, input_buffer
        
        # Threshold and normalize descriptor_vector
        threshold = norm(descriptor_vector) * descriptor_max_value
        descriptor_vector[descriptor_vector > threshold] = threshold
        descriptor_vector /= max(norm(descriptor_vector), float_tolerance)
        # Multiply by 512, round, and saturate between 0 and 255 to convert from float32 to unsigned char (OpenCV convention)
        descriptor_vector = round(512 * descriptor_vector)
        descriptor_vector[descriptor_vector < 0] = 0
        descriptor_vector[descriptor_vector > 255] = 255
        descriptors.append(descriptor_vector)
    return array(descriptors, dtype='float32')

In [4]:
# dma.register_map.S2MM_DMACR.RS = 1
# dma.register_map.S2MM_DMACR.RS = 0
# dma.register_map.S2MM_DMACR.Reset = 1
dma.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

In [5]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2

img1 = cv2.imread('dino_data/dino02.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
# keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
# print("Finished keypoints")
# keypoints = removeDuplicateKeypoints(keypoints)
# print("Finished remove duplicate keypoints")

Finished base_image
Finished num_octaves
Finished gaussian_kernels
Finished gaussian_images
Finished dog_images


In [6]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import cv2


keypoints_json = []
with open('data.json', 'r') as f:
    pck = json.load(f)
    for k in pck:
        keypoints_json.append(convertDictToKP(k))
# with open('data.json', 'w') as f:
#     json.dump(KeyPointsToJSON(keypoints), f)

# print(keypoints_json[0].octave)
# print(keypoints[0].octave)
kp1 = convertKeypointsToInputImageSize(keypoints_json)
print("Finished convert to image size")
des1 = generateDescriptors_hardware(keypoints_json, gaussian_images, dma)
print("Finished generate_descriptors")

Finished convert to image size
Generating descriptors
Continuing
Continuing
Continuing
Continuing
Continuing
Continuing
Continuing
Continuing
Continuing


/tmp/ipykernel_7656/77272442.py:55: RuntimeWarning: overflow encountered in ubyte_scalars
  dx = gaussian_image[window_row, window_col + 1] - gaussian_image[window_row, window_col - 1]
/tmp/ipykernel_7656/77272442.py:57: RuntimeWarning: overflow encountered in ubyte_scalars
  gradient_magnitude = sqrt(dx * dx + dy * dy)
/tmp/ipykernel_7656/77272442.py:56: RuntimeWarning: overflow encountered in ubyte_scalars
  dy = gaussian_image[window_row - 1, window_col] - gaussian_image[window_row + 1, window_col]


377798656
377786368


NameError: name 'fpga_unzip' is not defined

# Second Image Stuff

In [ ]:
img2 = cv2.imread('dino_data/dino05.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
print("Finished keypoints")
keypoints = removeDuplicateKeypoints(keypoints)
print("Finished remove duplicate keypoints")

In [ ]:
kp2 = convertKeypointsToInputImageSize(keypoints)
print("Finished convert to image size")
des2 = generateDescriptors_hardware(keypoints, gaussian_images, dma)
print("Finished generate_descriptors")

In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)


# Estimate homography between template and scene
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

M = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0]

# Draw detected template in scene image
h, w = img1.shape
pts = np.float32([[0, 0],
                    [0, h - 1],
                    [w - 1, h - 1],
                    [w - 1, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, M)

img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

h1, w1 = img1.shape
h2, w2 = img2.shape
nWidth = w1 + w2
nHeight = max(h1, h2)
hdif = int((h2 - h1) / 2)
newimg = np.zeros((nHeight, nWidth, 3), np.uint8)

for i in range(3):
    newimg[hdif:hdif + h1, :w1, i] = img1
    newimg[:h2, w1:w1 + w2, i] = img2

# Draw SIFT keypoint matches
for m in good:
    pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1] + hdif))
    pt2 = (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1]))
    cv2.line(newimg, pt1, pt2, (255, 0, 0))

plt.imshow(newimg)
plt.show()

In [ ]:
from sfm_algo_unpacked import localizeExtremumViaQuadraticFit, computeKeypointsWithOrientations, isPixelAnExtremum
from numpy import floor

def findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width, contrast_threshold=0.04):
    threshold = floor(0.5 * contrast_threshold / num_intervals * 255)
    keypoints = []

    for octave_index, dog_images_in_octave in enumerate(dog_images):
        for image_index, (first_image, second_image, third_image) in enumerate(zip(dog_images_in_octave, dog_images_in_octave[1:], dog_images_in_octave[2:])):
            for i in range(image_border_width, first_image.shape[0] - image_border_width):
                if (i % 10 == 0):
                    print('.', end='')
                if (i % 100 == 0):
                    print('')
                for j in range(image_border_width, first_image.shape[1] - image_border_width):
                    if isPixelAnExtremum(first_image[i-1:i+2, j-1:j+2], second_image[i-1:i+2, j-1:j+2], third_image[i-1:i+2, j-1:j+2], threshold):
                        localization_result = localizeExtremumViaQuadraticFit(i, j, image_index + 1, octave_index, num_intervals, dog_images_in_octave, sigma, contrast_threshold, image_border_width)
                        if localization_result is not None:
                            keypoint, localized_image_index = localization_result
                            keypoints_with_orientations = computeKeypointsWithOrientations(keypoint, octave_index, gaussian_images[octave_index][localized_image_index])
                            for keypoint_with_orientation in keypoints_with_orientations:
                                keypoints.append(keypoint_with_orientation)
    return keypoints